<img src="images/banner.png" style="width: 100%;">

# Working with Databases II Notebook 1

References:

[1] McKinney, Wes. *Python for data analysis.* " O'Reilly Media, Inc.", 2022.

[2] Teate, Renee MP. *SQL for Data Scientists: A Beginner's Guide for Building Datasets for Analysis.* John Wiley & Sons, 2021.

[3] Forta, Ben. *Sams Teach Yourself SQL in 10 Minutes a Day, 5th Edition*. O'Reilly Media, Inc., 2020

[4] Python sqlite3 documentation - https://docs.python.org/3/library/sqlite3.html

[5] Revised and grammar checked using ChatGPT - https://chatgpt.com/

Prepared by: Leodegario Lorenzo II

In the previous notebooks, we learned how to filter rows, select specific columns, and join multiple tables.
However, when presenting data, we rarely show every individual record. Instead, we summarize the data to highlight patterns, trends, and key metrics.

This is where **data aggregation** becomes essential.

In this notebook, we'll show and demonstrate the usage of the different aggregation functions normally available in SQL. We also show how to aggregate data in specific levels through the use of the `GROUP BY` clause.

We'll continue to use the farmer's market database as our database for examples.

In [48]:
import pandas as pd
import sqlalchemy as sqla

In [49]:
db = sqla.create_engine('sqlite:///data/farmers_market.db')

## 1 Aggregation Functions

SQL features a set of five *aggregate functions* listed by the table below:

| Function | Description |
| :------- | :---------- |
| `AVG()` | Return a column's average value |
| `COUNT()` | Return the number of rows in a column |
| `MAX()` | Returns a column's highest value |
| `MIN()` | Returns a column's lowest value |
| `SUM()` | Returns the sum of a column's values |

For a comprehensive list of available aggregation functions in SQLite, visit the following link - [https://sqlite.org/lang_aggfunc.html](https://sqlite.org/lang_aggfunc.html) 

The use of the aggregate functions is straightforward. We specify the column in which we want to describe using our desired aggregation function, then add it in our selection.

For example, if we want to know *What is the **average price** of all of the products in the farmer's market?* we can use the following query:

In [4]:
query = """
        SELECT
            AVG(original_price) average_price
        FROM vendor_inventory
        """
pd.read_sql(query, db)

,average_price
0,9.058896


We can also use aggreate functions in conjunction with filtering using `WHERE` statements. Let's say we want to know *How much is the **total sales of all transactions** in the farmer's market for the month of December 2019?*

In [5]:
query = """
        SELECT
            market_date,
            ROUND(SUM(quantity * cost_to_customer_per_qty), 2) AS total_sales
        FROM
            customer_purchases
        WHERE
            market_date BETWEEN '2019-12-01' AND '2019-12-31'
        """
pd.read_sql(query, db)

,market_date,total_sales
0,2019-12-04,5378.5


If we want to know *How **many transactions** were made during the month of December 2019?*, we can use the `COUNT()` or `COUNT(*)` function.

In [6]:
query = """
        SELECT
            market_date,
            ROUND(SUM(quantity * cost_to_customer_per_qty), 2) AS total_sales,
            COUNT(*) AS number_of_transactions
        FROM
            customer_purchases
        WHERE
            market_date BETWEEN '2019-12-01' AND '2019-12-31'       
        """
pd.read_sql(query, db)

,market_date,total_sales,number_of_transactions
0,2019-12-04,5378.5,222


The difference between `COUNT()` and `COUNT(*)` is that `COUNT(*)` counts the number of rows in a query including `NULL` values, meanwhile, `COUNT()` only considers non-`NULL` values. To demonstrate this, let's look at the `product` table:

In [7]:
query = """
        SELECT *
        FROM product
        """
pd.read_sql(query, db)

,product_id,product_name,product_size,product_category_id,product_qty_type
0,1,Habanero Peppers - Organic,medium,1,lbs
1,2,Jalapeno Peppers - Organic,small,1,lbs
2,3,Poblano Peppers - Organic,large,1,unit
3,4,Banana Peppers - Jar,8 oz,3,unit
4,5,Whole Wheat Bread,1.5 lbs,3,unit
5,6,Cut Zinnias Bouquet,medium,5,unit
6,7,Apple Pie,"10""",3,unit
7,8,Cherry Pie,"10""",3,unit
8,9,Sweet Potatoes,medium,1,lbs
9,10,Eggs,1 dozen,6,unit


Here, we see that the `product_qty_type` column has two `NULL` values, which will not be counted by the `COUNT()` function, but will be included in the computation of the number of rows or the `COUNT(*)` function.

In [8]:
query = """
        SELECT
            COUNT(product_qty_type) AS number_of_not_null_product_qty_types,
            COUNT(*) AS number_of_rows_in_product_table
        FROM product
        """
pd.read_sql(query, db)

,number_of_not_null_product_qty_types,number_of_rows_in_product_table
0,21,23


We can also use the `DISTINCT` before an aggregation to perform the summarization using only distinct values. This allows us to answer questions such as - *How **many booth types** are available in the farmer's market?*

In [9]:
query = """
        SELECT
            COUNT(DISTINCT booth_type)
        FROM booth
        """
pd.read_sql(query, db)

,COUNT(DISTINCT booth_type)
0,3


Note that here, the `NULL` values are once again not included in the count since we are using `COUNT()`.

## 2 Grouping Data

SQL becomes much more powerful when you use it to aggregate and analyze data in various level of summarization (through the use of `GROUP BY`).

From our previous notebook, we shown the following syntax, where we have the `GROUP BY` and `HAVING` clauses:

```sql
SELECT <columns to return>
FROM <table>
WHERE <conditional filter statements>
GROUP BY <columns to group on>
HAVING <conditional filter statements that run after grouping>
ORDER BY <columns to sort on>
```

Any of the aggregation functions in SQL can be used as functions for summarization using `GROUP BY`. As an example, let's say we want to know - *How **many products** has **each vendor** offered throughout the duration of the farmer's market?*

In [15]:
query = """
        SELECT
            vendor_id,
            COUNT(*) AS num_products_offered
        FROM vendor_inventory
        GROUP BY vendor_id
        ORDER BY num_products_offered DESC
        """
pd.read_sql(query, db)

,vendor_id,num_products_offered
0,8,426
1,7,301
2,4,70


Not that the above may not be an accurate answer to our query since it counts each row rather than each unique product per `vendor_id`, as such, the correct query should be:

In [16]:
query = """
        SELECT
            vendor_id,
            COUNT(DISTINCT product_id) AS num_products_offered
        FROM vendor_inventory
        GROUP BY vendor_id
        ORDER BY num_products_offered DESC
        """
pd.read_sql(query, db)

,vendor_id,num_products_offered
0,7,4
1,8,3
2,4,1


A pandas counterpart `groupby` method can also perform the same task.

In [20]:
query = """
        SELECT
            *
        FROM vendor_inventory
        """
vendor_inventory = pd.read_sql(query, db)

In [31]:
(vendor_inventory.groupby(['vendor_id']).product_id.nunique()
                 .sort_values(ascending=False)
                 .reset_index(name='num_products_offered'))

,vendor_id,num_products_offered
0,7,4
1,8,3
2,4,1


### Filtering Groups

Aside from being to perform data summarization across specified groups, SQL can also perform filtering after the aggregation using the `HAVING` clause.

For example, we want to know - *Which customers **have spent at least $3,000** at the farmers’ market since its inception?*

In [36]:
query = """
        SELECT
            customer_id,
            ROUND(SUM(quantity * cost_to_customer_per_qty), 2) AS total_sales
        FROM customer_purchases
        GROUP BY customer_id
        HAVING total_sales >= 3000
        ORDER BY total_sales DESC
        """
pd.read_sql(query, db)

,customer_id,total_sales
0,2,4179.45
1,5,3932.83
2,3,3832.16
3,4,3561.63
4,1,3530.92
5,11,3499.99
6,8,3403.68
7,12,3290.08
8,6,3016.47
9,9,3015.73


Notice the difference between `HAVING` and `WHERE`. The filtering of `WHERE` happens BEFORE the aggregation, which means, we CANNOT filter columns resulting from aggregation using `WHERE`. Whereas, the filtering of `HAVING` happens AFTER the aggregation, which means, we can filter columns resulting from aggregation using `HAVING`.

An example of the usage of `WHERE` and `HAVING` is answering questions like - *Which customers **have spent at least $300** at the farmers' market **during the month of December 2019**?*

In [38]:
query = """
        SELECT
            customer_id,
            ROUND(SUM(quantity * cost_to_customer_per_qty), 2) AS total_sales
        FROM customer_purchases
        WHERE
            market_date BETWEEN '2019-12-01' AND '2019-12-31'
        GROUP BY customer_id
        HAVING
            total_sales >= 300
        ORDER BY total_sales DESC
        """
pd.read_sql(query, db)

,customer_id,total_sales
0,24,588.0
1,22,324.0
2,5,320.5
3,3,308.0


### Performing Aggregations with Joined Tables

Finally, we also demonstrate that we can perform data aggregation with the use of joined tables. As an example, let's answer the question - *How much has **each customer** spent with **each vendor** (regardless of items purchased), for **totals at least $200** **during the month of December 2019**?*

Let's say also that we're presenting this to stakeholders of the farmers' market. As such, we also want a human readable output for our result.

In [47]:
query = """
        SELECT
            CONCAT(c.customer_first_name, ' ', c.customer_last_name) AS customer_name,
            v.vendor_name,
            ROUND(SUM(quantity * cost_to_customer_per_qty), 2) AS total_sales
        FROM customer_purchases AS cp
            LEFT JOIN customer AS c
                ON c.customer_id = cp.customer_id
            LEFT JOIN vendor AS v
                ON v.vendor_id = cp.vendor_id
        WHERE
            market_date BETWEEN '2019-12-01' AND '2019-12-31'
        GROUP BY cp.customer_id, cp.vendor_id
        HAVING total_sales >= 200
        ORDER BY total_sales DESC
        """
pd.read_sql(query, db)

,customer_name,vendor_name,total_sales
0,Dawn Nale,Annie`s Pies,588.0
1,George Rai,Annie`s Pies,324.0
2,Jane Connor,Annie`s Pies,278.0
3,Abigail Harris,Annie`s Pies,276.5
4,Bob Wilson,Annie`s Pies,232.0
5,Bonnie Hassan,Annie`s Pies,222.0
6,Deanna Washington,Annie`s Pies,212.5
7,Valerie Loftis,Annie`s Pies,204.5


<img src="images/banner-down.png" style="width: 100%;">